In [19]:
import pandas as pd
import numpy as np

# 1. 시카고 마라톤 원본 데이터 불러오기
chicago_df = pd.read_csv("./data/chicago_data.csv")

In [20]:
# 2. 구간 기록 컬럼명을 보스턴 데이터와 동일하게 통일
split_mapping = {
    '5km.time': '5K',
    '10km.time': '10K',
    '15km.time': '15K',
    '20km.time': '20K',
    'half.time': 'Half',
    '25km.time': '25K',
    '30km.time': '30K',
    '35km.time': '35K',
    '40km.time': '40K',
    'finish.time': 'Final_Time'
}
chicago_df = chicago_df.rename(columns=split_mapping)

In [21]:
# 3. 문자열 형태의 시간 데이터를 pandas timedelta로 변환
for col in split_mapping.values():
    chicago_df[col] = pd.to_timedelta(chicago_df[col], errors='coerce')

# timedelta 형식인 칼럼들 리스트
time_cols = ["5K", "10K", "15K", "20K", "Half", "25K", "30K", "35K", "40K", "Final_Time"]  

# 각 칼럼을 초 단위로 변환하여 새로운 컬럼 추가 (예: '5K_sec', '10K_sec' ...)
for col in time_cols:
    chicago_df[col] = pd.to_timedelta(chicago_df[col]).dt.total_seconds()


In [22]:
# 4. 공식 완주시간(Final_Time)을 시간 단위로 변환해 새로운 컬럼 생성
# chicago_df["Final_Time"] = chicago_df["Final_Time"].dt.total_seconds()

In [23]:
# 5. 연령대 그룹화 (20대, 30대, ...) - 보스턴 데이터와 통일된 형식

# def get_age_group(val):
#     try:
#         val = int(str(val)[:2])  # 예: '20-24' → 20
#         return val if val >= 20 else 20
#     except:
#         return np.nan

# chicago_df["Age group"] = chicago_df["age_class"].map(get_age_group)

In [24]:
# 6. 성별 표기 통일: man/woman → M/F
chicago_df["M/F"] = chicago_df["gender"].map({"man": "0", "woman": "1"})

In [25]:
# 7. 각 구간별 페이스 계산 (분/km 기준)
dist_cols = ["5K", "10K", "15K", "20K", "Half", "25K", "30K", "35K", "40K"]
dists = [5, 10, 15, 20, 21.0975, 25, 30, 35, 40]

for col, dist in zip(dist_cols, dists):
    chicago_df[col + "p"] = chicago_df[col] / (dist)

split_mapping = {
    '5Kp': '5p',
    '10Kp': '10p',
    '15Kp': '15p',
    '20Kp': '20p',
    'Halfp': 'Halfp',
    '25Kp': '25p',
    '30Kp': '30p',
    '35Kp': '35p',
    '40Kp': '40p',
}
chicago_df = chicago_df.rename(columns=split_mapping)

In [26]:
# chicago_df.info()


In [27]:
# 8. '.time_of_day'로 끝나는 모든 컬럼 제거
cols_to_drop = [col for col in chicago_df.columns if col.endswith(".time_of_day")]
chicago_df.drop(columns=cols_to_drop, inplace=True)

def age_group_by_upper(val):
    try:
        return int(str(val).split('-')[1])  # '20-24' → 24
    except:
        return np.nan

# Age_class 결측값 제거 및 전처리
chicago_df = chicago_df[~(chicago_df["age_class"].isna() | (chicago_df["age_class"] == "MT53"))]
chicago_df['age_class'] = chicago_df['age_class'].map(age_group_by_upper)



In [28]:
chicago_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26950 entries, 0 to 26999
Data columns (total 31 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           26950 non-null  object 
 1   gender         26950 non-null  object 
 2   country        26950 non-null  object 
 3   bib            0 non-null      float64
 4   age_class      26861 non-null  float64
 5   finish_time    26950 non-null  object 
 6   place_gender   26950 non-null  int64  
 7   place_overall  26950 non-null  int64  
 8   details_url    26950 non-null  object 
 9   city_state     26927 non-null  object 
 10  start.time     26950 non-null  object 
 11  5K             26805 non-null  float64
 12  10K            26912 non-null  float64
 13  15K            26916 non-null  float64
 14  20K            26918 non-null  float64
 15  Half           26920 non-null  float64
 16  25K            26921 non-null  float64
 17  30K            26922 non-null  float64
 18  35K        

In [29]:
# 피처명 수정
chicago_df.rename(columns={"bib": "Bib", "name": "Name", "place_gender": "Gender", "place_overall": "Overall", "country": "Country"}, inplace=True)


In [30]:
# 나이(Age)를 기준으로 연령대 그룹(구간)으로 나누는 함수 ex. 0~19 -> 20 / 20~29 -> 30
# def group_calculator(serie):
#     list=[]
#     for i in range(len(serie)):
#         if serie[i]<20:
#             list.append(20)
#         elif serie[i]<30:
#             list.append(30)
#         elif serie[i]<40:
#             list.append(40)
#         elif serie[i]<60:
#             list.append(60)
#         elif serie[i]<80:
#             list.append(80)
#         else:
#             list.append(100)
    
#     return list
def group_calculator(val):
    try:
        val = float(val)
        if val <= 19:
            return "19"
        elif val <= 24:
            return "24"
        elif val <= 29:
            return "29"
        elif val <= 34:
            return "34"
        elif val <= 39:
            return "39"
        elif val <= 44:
            return "44"
        elif val <= 49:
            return "49"
        elif val <= 54:
            return "54"
        elif val <= 59:
            return "59"
        elif val <= 64:
            return "64"
        elif val <= 69:
            return "69"
        else:
            return "70"
    except:
        return np.nan

    
# 연령대 그룹 (20, 30, 40, 60, 80, 100) 생성 후, Age group 컬럼 생성
# chicago_df['Age group']=pd.Series(group_calculator(chicago_df['age_class']))
chicago_df['Age_group'] = chicago_df['age_class'].map(group_calculator)

In [31]:

# 완주시간 결측값 제거
chicago_df = chicago_df[chicago_df["Final_Time"].notna()]

chicago_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 26928 entries, 0 to 26999
Data columns (total 32 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         26928 non-null  object 
 1   gender       26928 non-null  object 
 2   Country      26928 non-null  object 
 3   Bib          0 non-null      float64
 4   age_class    26861 non-null  float64
 5   finish_time  26928 non-null  object 
 6   Gender       26928 non-null  int64  
 7   Overall      26928 non-null  int64  
 8   details_url  26928 non-null  object 
 9   city_state   26927 non-null  object 
 10  start.time   26928 non-null  object 
 11  5K           26805 non-null  float64
 12  10K          26912 non-null  float64
 13  15K          26916 non-null  float64
 14  20K          26918 non-null  float64
 15  Half         26920 non-null  float64
 16  25K          26921 non-null  float64
 17  30K          26922 non-null  float64
 18  35K          26923 non-null  float64
 19  40K      

In [32]:
# 'Pace' 칼럼 추가 - 러닝 전체 평균 pace

# # 우선 Final_Time을 시간 문자열(hh:mm:ss) → 분 단위 float 값으로 변환
# chicago_df["Final_Time (min)"] = pd.to_timedelta(chicago_df["Final_Time"]).dt.total_seconds() / 60

# # 전체 마라톤 거리로 나눠 평균 pace (분/km) 계산
# chicago_df["Pace"] = chicago_df["Final_Time (min)"] / 42.195

# def format_pace(pace_float):
#     minutes = int(pace_float)
#     seconds = int(round((pace_float - minutes) * 60))
#     return f"{minutes}:{seconds:02d}"

# # 'pace' 컬럼을 문자열 형식으로 변환해서 새로운 컬럼으로 저장
# chicago_df["Pace"] = chicago_df["Pace"].apply(format_pace)



In [33]:
chicago_df = chicago_df.drop(columns=['gender', 'age_class', 'Gender', 'Overall', 'city_state', 'details_url', 'start.time', 'Name', 'finish_time'])


chicago_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26928 entries, 0 to 26999
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Country     26928 non-null  object 
 1   Bib         0 non-null      float64
 2   5K          26805 non-null  float64
 3   10K         26912 non-null  float64
 4   15K         26916 non-null  float64
 5   20K         26918 non-null  float64
 6   Half        26920 non-null  float64
 7   25K         26921 non-null  float64
 8   30K         26922 non-null  float64
 9   35K         26923 non-null  float64
 10  40K         26923 non-null  float64
 11  Final_Time  26928 non-null  float64
 12  M/F         26928 non-null  object 
 13  5p          26805 non-null  float64
 14  10p         26912 non-null  float64
 15  15p         26916 non-null  float64
 16  20p         26918 non-null  float64
 17  Halfp       26920 non-null  float64
 18  25p         26921 non-null  float64
 19  30p         26922 non-null  fl

In [34]:
# 목표 그룹 함수 정의
def time_group(t):
    if t < 10800.0:
        return "3"
    elif t < 14400.0:
        return "4"
    elif t < 18000.0:
        return "5"
    elif t < 21600.0:
        return "6"
    else:
        return "7"

# 그룹 컬럼 생성
chicago_df['Sub'] = chicago_df['Final_Time'].apply(time_group)

In [35]:
import numpy as np

# 총 행 수
num_rows = len(chicago_df)

# 중복 없이 무작위 숫자 생성 (예: 100000 ~ 999999 사이)
chicago_df["Bib"] = np.random.choice(range(1, num_rows+1), size=num_rows, replace=False)

chicago_df["Dataset"] = "C"

chicago_df["Year"] = 2021



chicago_df = chicago_df[["Bib", "Age_group", "M/F", "Country", "5p", "10p", "15p", "20p", "25p", "30p", "35p", "40p", "Final_Time", "Sub", "5K", "10K", "15K", "20K", "25K", "30K", "35K", "40K", "Dataset", "Year"]]


In [36]:
# 전처리 완료된 파일 저장
chicago_df.to_csv("./data/chicago_data_processed.csv", index=False)
print("[✓] chicago_data_processed.csv 저장 완료")

[✓] chicago_data_processed.csv 저장 완료
